# **Unsupervised Learning**

**This notebook has been prepared by:**
* Zizipho Tyeko
* Siyamanga Malawu
* Lejone Malokosta
* Pfano Phungo
* Mogau Mogashoa
* Dunyiswa Matshaya

### How is the notebook going to work?

This notebook is a layout of a recommender system that is used to predict a movie user possible rating. The notebook will make use of the recommender system methods and techniques using sequential steps to get to the prediction of the possible expected results.


# **Movie Recommendation Challenge**

## **Recommender System**

Recommender systems are amid the most well known applications of data science today. They are used to predict the "rating" or "preference" that a user would possibly give to an item. Recommender systems uses its techniques by searching through large volume of dynamically generated information to provide users with personalized content and services.
Technically recommender system has the ability to predict whether a particular user would prefer an item or not based on the user’s profile.

# **Introduction**

The aim of this notebook is to predict how a user will rate a movie they have not yet viewed, based on their historical preference on a movie website or application e.g Netflix, Showmax or Amazon Prime.

Movie websites and applications can improve their reliability and enhance their customer experience by providing an estimated rating or preference of a movie through a recommender system used to model the predicted results.

Recommender systems are essential economically and socially in today's technology driven world. This can help movie companies in ensuring that their users can make the appropriate choices surrounding the content that they regulary engage with.

## **Aim**
Design a recommendersystem which will predict a user possible rating on a movie that they have not viewed yet based on they user history of their movie ratings.

## **Scope**
The scope of this project is to analyse and search through large volume of dynamically generated information consisting of movie ratings given by a user and information describing the movie.
These ratings will be used to train machine learning models to help with the prediction of the ratings given by a user on an unseen movie. This could also help with providing users with personalised content and services.

<img src="https://posteet.com/wp-content/uploads/2019/11/movies.png" width=90%>

# **Table of Content**

1. Import packages
2. Loading Datasets
3. Data Description
4. Explanotory Data Analysis
5. Data Filtering
6. Varibale Selection
7. Modeling
8. Model Comparison
9. Model Explanation
10. Submission
11. Application Pickled files


## 1. Importing packages

In [ ]:
pip install comet_ml

In [ ]:
# Import comet_ml in the top of your file
import comet_ml
from comet_ml import Experiment

experiment = Experiment(api_key="PXaqoLTlHzWF85kimEv0zHJPz",
                        project_name="unsupervised-predict", workspace="lejone")

In [ ]:
# utilities
import numpy as np
import pandas as pd

#pre-processing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#plotting
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
import cufflinks as cf
plt.style.use('ggplot')
%matplotlib inline
sns.set()

from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, SVDpp
from surprise.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb

from sklearn.metrics import mean_squared_error
from datetime import datetime
import datetime
from timeit import default_timer

#others
import re

In [ ]:
# Context manager allows logging og parameters

experiment.context_manager("validate")

In [ ]:
# URL where experiments can be found

experiment.url

# 2. Loading Datasets

This section loads and checks the data needed to run this notebook.
To ensure the notebook runs without any error, the relevent csv files need to be present in the notebook's directory before running the next cell.

In [ ]:
train = pd.read_csv('../input/edsa-recommender-system-predict/train.csv') 
test = pd.read_csv('../input/edsa-recommender-system-predict/test.csv')
scores = pd.read_csv('../input/edsa-recommender-system-predict/genome_scores.csv')
tags = pd.read_csv('../input/edsa-recommender-system-predict/genome_tags.csv')
imdb = pd.read_csv('../input/edsa-recommender-system-predict/imdb_data.csv') 
links = pd.read_csv('../input/edsa-recommender-system-predict/links.csv') 
movies = pd.read_csv('../input/edsa-recommender-system-predict/movies.csv')
sample = pd.read_csv('../input/edsa-recommender-system-predict/sample_submission.csv')

In [ ]:
print("Train\n")
print(train.head(5))
print("Test\n")
print(test.head(5))
print("Scores\n")
print(scores.head(5))
print("Tags\n")
print(tags.head(5))
print("imdb\n")
print(imdb.head(5))
print("Links\n")
print(links.head(5))
print("\nMovies")
print(movies.head(5))

print("\n(Datasets were imported correctly)")

# 3. Data Description

Now that we have loaded the datasets into the notebook here we display the information of the datasets that we will be using, check if there are gaps or blanks in the datasets and check how the sentyiments of the rating look like.

### 3.1 Checking the data types of the train data sets and the unique values of the rating column in the train dataset

In [ ]:
print(train.info())
print('The rating sentiments are as follows: /n')
print(train['rating'].unique())

The train dataset consist of the user rating information, which is the user ID, the movie the user rated using the movie ID, the timestamp that the rating was made by the user and the rating itself. 
The rating sentiment ranges from 0.5 to 5. With 0.5 being the lowest rating and 5 being the highest rating of a movie. 

### 3.2 Checking and removing null values from the datasets that we are planning to use

In [ ]:
print("No of Nan values in our dataframe : ", sum(train.isnull().any()))

In [ ]:
print("No of Nan values in our dataframe : ", sum(test.isnull().any()))

In [ ]:
print("No of Nan values in our dataframe : ", sum(movies.isnull().any()))

The above datasets are the ones that we will be using the most in this notebook. So we are checking if there is any null values that we would need to remove from the datasets. However reading from the results above there are no null values from the datasets at all. So because of that we won't need to remove any null values.

### 3.3 Checking and removing duplicate values

In [ ]:
# Remove duplicate entries for userId, movieId and rating in the train dataset 
dup_bool = train.duplicated(['userId','movieId','rating'])
dups = sum(dup_bool) 
print("There are {} duplicate rating entries in the data..".format(dups))

In [ ]:
# Remove duplicate entries for userId, movieId in the test dataset 
dup_bool = test.duplicated(['userId','movieId'])
dups = sum(dup_bool) 
print("There are {} duplicate rating entries in the data..".format(dups))

In [ ]:
# Remove duplicate entries for movieId, title, genres in the movies dataset 
dup_bool = movies.duplicated(['movieId', 'title', 'genres'])
dups = sum(dup_bool) 
print("There are {} duplicate rating entries in the data..".format(dups))

We also checking for any duplicated data in the above datasets. As we we can see we have 0 duplicated data, so no removal of any duplicates from the datasets

# **4. Exploratory Data Analysis**

### 4.1 Calculating the number of user's, movies and ratings from the train dataset

In [ ]:
'''box plot for ratings'''
box = train['rating']
plt.boxplot(box)
plt.ylabel('Ratings')
plt.show()

The above box plot is illustrating the 5 number summary of the ratings in the train dataset. 75% of the ratings are above 3 this means that people like the movies on this dataset.

In [ ]:
print("Total data ")
print("-"*50)
print("\nTotal no of ratings :",train.shape[0])
print("Total No of Users   :", len(np.unique(train.userId)))
print("Total No of movies  :", len(np.unique(train.movieId)))

The above code is showing us the total length of the features in the train dataset.
This is the total number of the ratings, the total number of users and the total number of movies rated.

Below we will use a method called "human". This method will just help us with making one of our graphs more readable

In [ ]:
'''This is a method that improves the readability of a graph.'''
def human(num, units = 'M'):
    units = units.lower()
    num = float(num)
    if units == 'k':
        return str(num/10**3) + " K"
    elif units == 'm':
        return str(num/10**6) + " M"
    elif units == 'b':
        return str(num/10**9) +  " B"

In [ ]:
fig, ax = plt.subplots()
plt.title('Distribution of ratings over Training dataset', fontsize=15)
sns.countplot(train.rating)
ax.set_yticklabels([human(item, 'M') for item in ax.get_yticks()])
ax.set_ylabel('No. of Ratings(Millions)')

plt.show()

In the above graph we can see that we have over 2.5 million 4.0 scale number of ratings follwed by the 3.0 scale that has a little below 2 million number of ratings and in 3rd place it is the 5.0 scale that has a little below 1.5 million ratings. 

### 4.2 Analysis of ratings of a movie given by a user

To analyse the ratings given by a user for a specific movie, we will use the movieId and rating features from the Train dataset

In [ ]:
train_df = train.copy()

In [ ]:
no_of_ratings_per_movie = train_df.groupby(by='movieId')['rating'].count().sort_values(ascending=False)

fig = plt.figure(figsize=plt.figaspect(.5))
ax = plt.gca()
plt.plot(no_of_ratings_per_movie.values)
plt.title('Number of ratings per Movie')
plt.xlabel('Movie')
plt.ylabel('No of Users who rated a movie')
ax.set_xticklabels([])

plt.show()

The plot is very skewed.. just like number of ratings given per user.

There are some movies which are very popular which are rated by a huge number of users, though they are very few (less than 15 %).

But most of the movies(about 85%) have ratings below 1000.

### 4.3 Converting timestamp column into date time

Below we are converting the timestamp feature in the T
rain dataset from an integer datatype to a datetime datatype so that we can see the actual date of the rating, the actual month, the day of the week and the year that the rating was given by the user.

In [ ]:
'''converting timestamp column to datetime'''
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], unit='ms')
train_df.head()

#### 4.3.1 Adding a day of week column in the train data set to analyse ratings on each day of the week

In [ ]:
'''extracting days of the week'''
train_df['day_of_week'] = train_df['timestamp'].dt.dayofweek
days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
train_df['day_of_week'] = train_df['day_of_week'].apply(lambda x: days[x])
train_df.tail()

In [ ]:
'''plotting average number of ratings for each day'''
train_df['day_of_week'].value_counts().plot(kind='barh')
plt.xlabel('Average number of ratings')
plt.ylabel('Days of the week')

We calcated the average rating for each day of the week and we can see that we receive the most ratings on a Sunday and on a Saturday. As that would be the time that user's are more settled at home, because it would be end of week and thats the time that most user's would have time to watch movies.

### 4.4 Analysis of ratings given by a user

In the following we will analyse the ratings that are made by the users. We use the userId and the rating feature from the Train dataset to do this analysis.

In [ ]:
no_of_rated_movies_per_user = train_df.groupby(by='userId')['rating'].count().sort_values(ascending=False)

no_of_rated_movies_per_user.head()

Above we can see the total number of ratings of the top 5 users.

In [ ]:
'''plotting the PDF and CDF graphs'''
fig = plt.figure(figsize=plt.figaspect(.5))

ax1 = plt.subplot(121)
sns.kdeplot(no_of_rated_movies_per_user, shade=True, ax=ax1)
plt.xlabel('No of ratings by user')
plt.title("PDF")

ax2 = plt.subplot(122)
sns.kdeplot(no_of_rated_movies_per_user, shade=True, cumulative=True,ax=ax2)
plt.xlabel('No of ratings by user')
plt.title('CDF')

plt.show()

Most users seem to have low ratings. This is evident from the PDF plot above as number of ratings by user is very low after 2000 ratings. There also seems to be some more users with ratings around 10 000 and less. We now have to investigate the distribution of the ratings further.

In [ ]:
no_of_rated_movies_per_user.describe()

In [ ]:
quantiles = no_of_rated_movies_per_user.quantile(np.arange(0,1.01,0.01), interpolation='higher')

In [ ]:
plt.title("Quantiles and their Values")
quantiles.plot()
'''quantiles with 0.05 difference'''
plt.scatter(x=quantiles.index[::5], y=quantiles.values[::5], c='orange', label="quantiles with 0.05 intervals")
# quantiles with 0.25 difference
plt.scatter(x=quantiles.index[::25], y=quantiles.values[::25], c='m', label = "quantiles with 0.25 intervals")
plt.ylabel('No of ratings by user')
plt.xlabel('Value at the quantile')
plt.legend(loc='best')

# annotate the 25th, 50th, 75th and 100th percentile values....
for x,y in zip(quantiles.index[::25], quantiles[::25]):
    plt.annotate(s="({} , {})".format(x,y), xy=(x,y), xytext=(x-0.05, y+500)
                ,fontweight='bold')


plt.show()

Below we see that most user ratings are higher on the 75th percentile

In [ ]:
quantiles[::5]

How many ratings do we have at the last 5% of all ratings??

In [ ]:
print('\n No of ratings at last 5 percentile : {}\n'.format(sum(no_of_rated_movies_per_user>= 222)) )

As per our observation earlier, number of ratings at last 5 percentile is 8182 out of 162541 ratings. The mean ratings is 49.218536 with standard deviation of 86.009691.

### 4.5 Calculating the number of movies per genre

Which genre appears the most with the rated movies?

Below we will look at the genres feature from the Movies dataset and see which genre seems to appear the most with the rated movies. 

We made use of the count_genre function that will help us count the number of times each genre is mentioned

In [ ]:
movies_df = movies.copy()

In [ ]:
'''Split the genres into individual genres'''
genre_labels = set()
for s in movies_df['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))
    
'''Function that counts the number of times each of the genre appear'''
def count_genre(dataset, ref_col, census):
    genre_count = dict()
    for s in census: 
        genre_count[s] = 0
    for census_keywords in dataset[ref_col].str.split('|'):        
        if type(census_keywords) == float and pd.isnull(census_keywords): 
            continue        
        for s in [s for s in census_keywords if s in census]: 
            if pd.notnull(s): 
                genre_count[s] += 1
    '''______________________________________________________________________
     convert the dictionary in a list to sort the keywords by frequency '''
    genre_freq = []
    for k,v in genre_count.items():
        genre_freq.append([k,v])
    genre_freq.sort(key = lambda x:x[1], reverse = True)
    return genre_freq, genre_count

'''Calling this function gives access to a list of genre keywords which are sorted by decreasing frequency'''
genre_freq, dum = count_genre(movies_df, 'genres', genre_labels)
genre_freq[:5]

In [ ]:
'''Convert the frequency to dictionary'''
genres = dict()
trunc_freq = genre_freq[0:18]
for s in trunc_freq:
    genres[s[0]] = s[1]
    
    '''Convert to Dataframe'''
df=pd.DataFrame.from_dict(genres, orient='index').transpose()
df

In [ ]:
genres_list = list(genres.keys())

'''Plot a bar plot of genres sorted in ascending order'''
plt.figure(figsize=(12,7))
genre_counts = df.loc[:,genres_list].sum().sort_values(ascending=False)
sns.barplot(x=genre_counts.index, y=genre_counts.values)
plt.xticks(rotation=60)
plt.xlabel('Genre')
plt.ylabel('Number of movies');

In the above histogram we can see that the Drama genre seems to be having the most movies tags with a little over 25000 released movies followed by the Comedy genre that has over over 15000 movies. This could possibly mean that most ratings are coming from these two genres as well.

#### 4.5.1 Using WordCloud to check which words appear most frequent under genres

We want use the WordCloud technique to read into our genres to see which words is commonly found in each genre of a movie. 
This will help us confirm the above graph as well, because if a movie genre appears the most then that will mean that it is tagged the most in the rated movies.
We will make use of the Movies and the Tags datasets as it will help us analyse the the data we need using the genres feature from the Movies dataset and the tag feature from the Tag dataset.

In [ ]:
movies['genres'] = movies['genres'].astype('str')
tags['tag'] = tags['tag'].astype('str')
overview_corpus = ' '.join(movies['genres'])
tag = ' '.join(tags['tag'])

In [ ]:
overview_wordcloud = WordCloud(stopwords=STOPWORDS, background_color='white', height=2000, width=4000).generate(overview_corpus)
plt.figure(figsize=(16,8))
plt.imshow(overview_wordcloud)
plt.axis('off')
plt.show()

The wordcloud confirms the genre graph above where Drama and Comedy seems to appear to have the most movies. The wordcloud shows that the Drama and Comedy words appear the most or are most common on the movie genres and seems to be tagged more than other genres as well. These two genres are also followed by the Romance and Thriller genres that also seem to be common words on the WordCloud.

### 4.6 Calculating the number of movies released each year and number of ratings each year

Before we can run any graphs we first need to do the following:

* Split title and year into separate columns. Convert year to datetime.
* Categorize genres properly: split strings into boolean columns per genre.
* Modify the rating timestamp: from universal seconds to datetime year.
* Check for NaN values. Clean (delete rows) if % of NaN values is small.

We are going to use the Train and the Movies datasets. As we want to view the number of movies released on a yearly basis and also take a look at the ratings on a yearly basis as well

In [ ]:
'''Using the timestamp feature to generate the year and the month of the ratings'''

train_yr_mnth =  train.copy()


train_yr_mnth['year'] = train_yr_mnth['timestamp'].apply(lambda timestamp: datetime.fromtimestamp(timestamp).year)
train_yr_mnth['month'] = train_yr_mnth['timestamp'].apply(lambda timestamp: datetime.fromtimestamp(timestamp).month)
train_yr_mnth.head()

In [ ]:
st = default_timer()

'''We are using the movieId, title, genre and rating features from the Movies and Train datasets'''
    
'''Organise a bit and store into feather-format'''
movies_df1.sort_values(by='movieId', inplace=True)
movies_df1.reset_index(inplace=True, drop=True)
train_df1.sort_values(by='movieId', inplace=True)
train_df1.reset_index(inplace=True, drop=True)

print(train_df1.dtypes)

'''Split title and release year in separate columns in movies dataframe. Convert year to timestamp.'''
movies_df1['year'] = movies_df1.title.str.extract("\((\d{4})\)", expand=True)
movies_df1.year = pd.to_datetime(movies_df1.year, format='%Y')
movies_df1.year = movies_df1.year.dt.year 
movies_df1.title = movies_df1.title.str[:-7]

'''Categorize movies genres properly. '''
genres_unique = pd.DataFrame(movies_df1.genres.str.split('|').tolist()).stack().unique()
genres_unique = pd.DataFrame(genres_unique, columns=['genre']) # Format into DataFrame to store later
movies_df1 = movies_df1.join(movies_df1.genres.str.get_dummies().astype(bool))
movies_df1.drop('genres', inplace=True, axis=1)

# Modify rating timestamp format (from seconds to datetime year)
#ratings.timestamp = pd.to_datetime(ratings.timestamp, unit='s')
#train_df1.timestamp = pd.to_datetime(train_df1.timestamp, infer_datetime_format=True)
#train_df1.timestamp = train_df1.timestamp.dt.year


print ("Number of movies Null values: ", max(movies_df1.isnull().sum()))
print ("Number of ratings Null values: ", max(train_df1.isnull().sum()))
movies_df1.dropna(inplace=True)
train_df1.dropna(inplace=True)
    
'''Organise a bit, then save into feather-formatand clear from memory'''
movies_df1.sort_values(by='movieId', inplace=True)
train_df1.sort_values(by='movieId', inplace=True)
movies_df1.reset_index(inplace=True, drop=True)
train_df1.reset_index(inplace=True, drop=True)
    
runtime = default_timer() - st
print ("Elapsed time(sec): ", round(runtime,2))

Let's plot the graph for the above data that has been processed. 

In [ ]:
st = default_timer()

'''Plotting the graph of the above processed data'''
dftmp = movies_df1[['movieId', 'year']].groupby('year')

fig, ax1 = plt.subplots(figsize=(10,5))
ax1.plot(dftmp.year.first(), dftmp.movieId.nunique(), "g-o")
ax1.grid(None)
ax1.set_ylim(0,)

dftmp = train_df1[['rating', 'year']].groupby('year')
ax2 = ax1.twinx()
ax2.plot(dftmp.year.first(), dftmp.rating.count(), "r-o")
ax2.grid(None)
ax2.set_ylim(0,)

ax1.set_xlabel('Year')
ax1.set_ylabel('Number of movies released'); ax2.set_ylabel('Number of ratings')
plt.title('Movies per year')
plt.show()

# Housekeeping
%reset_selective -f (^dftmp$|^ax1$|^ax2$)

runtime = default_timer() - st
print ("Elapsed time(sec): ", round(runtime,2))

Number of movies released per year increases almost exponentially until 2017, then flattening and dropping signifincantly thereafter each year. This could possibly mean that expontential growth (i.e. bubbles) is seldom sustainable in the long term.
The ratings given by users only started in the year 1995 up until 2019. The ratings has been changing a lot some years the ratings increased some decreased but there has been a tremendous decrease in 2009 but then a tremendous increase in 2015 and then there after the ratings keep slightly decreasing each year.

In the graph below we are going to look at the number of overall ratings on a monthly basis

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,5))
dftmp1 = train_df1[['rating', 'month']].groupby('month')
ax2 = ax1.twinx()
ax1.plot(dftmp1.month.first(), dftmp1.rating.count(), "r-o")
ax2.grid(None)
ax2.set_ylim(0,)

ax1.set_xlabel('Month')
#ax1.set_ylabel('Number of movies released'); ax2.set_ylabel('Number of ratings')
ax1.set_ylabel('Number of ratings')
plt.title('Movies per month')
plt.show()

The above graph is showing that most ratings are received from the month of September up until November in a year, and then start decreasing in the month of December. The decrease is not significant, but that is maybe because users are busy with holiday commitments during that time.

### 4.7 Insights on the imbd features

#### 4.7.1 Lets have a look at the budget feature of the imbd dataset

The imdb dataset seem to give us more insight with regards to the movies. We will be using both the train and the imdb dataset to see what the yearly budget was for the movies.

In [ ]:
imdb_df = imdb.copy()
imdb_df.head()

'''removed special characters from the budget column and change its data type to float data type'''

imdb_df['budget'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]',value=r'') 
imdb_df['budget'] = imdb_df['budget'].astype(float)

'''merging the imdb and train table to see the overall yearly budget for all the movies'''
budget_data = pd.merge(imdb_df, train_df1, on='movieId')

year_mean = budget_data.groupby('year').mean()

year_mean[['budget']].plot(title = 'Yearly Movie Budget',color=('DarkBlue'),linestyle=('-'),figsize=(10, 8))
plt.xlabel('Year')
plt.ylabel('Budget');

In the above graph we can see that the budget of the movies increases every year. This could be due to change of time, technology and other contributing factors that cost more on a yearly basis as time goes. So that would lead to cost of production costing more each year

#### **4.7.2.Runtime of movies**

In [ ]:
runtime = pd.merge(imdb, movies, on='movieId')
runtime.head(2)

In [ ]:
#top 5 movies with longest running time
runtime['runtime'] = runtime['runtime'].astype(float)
b = runtime.drop(['movieId','title_cast', 'director', 'budget', 'plot_keywords', 'genres'], axis=1)
b.head()
b.nlargest(5,['runtime'])

The longest movie is Taken (2002)

Here we see that people describe movies using words that are not too specific eg hero. Most people just use simple words like good or best.

#### 4.7.3 Directors with most ratings

In [ ]:
#directors whose movies are the most rated
b = pd.merge(train, imdb, on='movieId')
mean_count = pd.DataFrame(b.groupby('director')['rating'].mean())
mean_count['rating_counts'] = pd.DataFrame(b.groupby('director')['rating'].count())
mean_count.nlargest(5,['rating_counts'])

The director with most ratings is Quentin Tarantino with an average rating of 3.9

# Movie budgets

In [ ]:
#Top 5 movies with highest budget
budget = pd.merge(movies, imdb, on='movieId')
budget['budget'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]',value=r'') 
budget['budget'] = budget['budget'].astype(float)
z = budget.drop(['movieId','title_cast', 'director', 'runtime', 'plot_keywords', 'genres'], axis=1)
z.head()
z.nlargest(5,['budget'])

The movie with the highest budget is My Way (Mai Wei) at 30 billion dollars

In [ ]:
#merging the imdb and train table to see the overall yearly budget for all the movies
budget_data = pd.merge(budget, train_yr_mnth, on='movieId')
budget_data.head()

In [ ]:
year_mean = budget_data.groupby('year').mean()

In [ ]:
year_mean[['budget']].plot(title = 'Yearly Movie Budget',color=('DarkBlue'),linestyle=('-'),figsize=(10, 8))
plt.xlabel('Year')
plt.ylabel('Budget');

The above graph is illastrating that the budget of the movies increases every year. This could be due to change of time, technology and other contributing factors that cost more on a yearly basis as time goes.

In [ ]:
#Merging the train and the imdb table to analyse any patterns between these two datasets regarding the movie rating and movie budget
budget_rating = pd.merge(train, imdb_df, on='movieId')
budget_rating.head()

In [ ]:
budget_rating.plot.scatter(x='rating', y='budget',title='Budget vs Rating Avg', color='DarkBlue', figsize=(6,5))
plt.xlabel('Ratings')
plt.ylabel('Budget');



The above graph is just showing us that the ratings on a movie does not really depend much on the budget of a movie. So a movie can have a high budget but still not get much ratings from user's.


# Modelling

### Two types of Recommender System

- Collaborative Filtering Recommender System
- Content-Based Recommender System

#### Collaborative Filtering Recommender System
Collaborative filtering recommender systems are based on the past interactions recorded between users and items in order to produce new recommendations. These interactions are stored in the so-called “user-item interactions matrix”.

#### Advantages of Colaborative Filtering Recommender System
- Works for any kind of item since no feature selection is needed
- Requires not content analysis & extraction
- Independent of any machine-readable representation of the objects being recommended
- More diverse and serendipitous recommendation




#### Content-Based Recommender System
Content based recommender system use additional information about users and/or items to make predictions.This additional information can be, for example, the age, the sex, the job or any other personal information for users.

#### Advantages of Content-Based Recommender Sytem
- Content representations are varied and they open up the options to use different approaches like: text processing techniques, the use of semantic information, inferences, etc…
- It is easy to make a more transparent system: we use the same content to explain the recommendations.
- We can avoid the “new item problem”
#### Disadvantages of Content-Based Recommender Sytem
- Content-Based RecSys tend to over-specialization: they will recommend items similar to those already consumed, with a tendecy of creating a “filter bubble”.

We implemented a collaborative recommendation system. The methods based on Collaborative Filtering have shown to be, empirically, more precise when generating recommendations

In [ ]:
#taking a sample of 5% of the training data 
train = train.sample(frac =.05)

We had to take a sample of the data for traing in this notebook due to insufficient space here in kaggle to run a bigger fraction of the data.

In [ ]:
"""
   This code is to build the trainset from train data. The reader() is used to read the file,
   which returns an iterable reader object. The new data frame 'data' is to specify how to read the dataframe.
   The build_full_trainset() is to format the data for use in the model fitting
"""

#reader() is used to read the file, which returns an iterable reader object.
reader = Reader(rating_scale=(0.5,5.0))
# It is to specify how to read the dataframe.
data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)
# build the trainset from train data.., It is of dataset format from surprise library..
trainset = data.build_full_trainset()

# Fitting models

## Singular Value Decomposition

The Singular Value Decomposition (SVD), a method from linear algebra that has been generally used as a dimensionality reduction technique in machine learning. SVD is a matrix factorisation technique, which reduces the number of features of a dataset by reducing the space dimension from N-dimension to K-dimension (where K<N). In the context of the recommender system, the SVD is used as a collaborative filtering technique.

- __ Predicted Rating : __
    - 
    - $ \large  \hat r_{ui} = \mu + b_u + b_i + q_i^Tp_u $
    
        - $\pmb q_i$ - Representation of item(movie) in latent factor space
        
        - $\pmb p_u$ - Representation of user in new latent factor space

- __Optimization problem with user item interactions and regularization (to avoid overfitting)__
    - 
    - $\large \sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
\lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right) $

In [ ]:
#fitting the model using the SVD algorithm with the prepared trainset data from above chuck of code.
svd = SVD()
svd.fit(trainset)

### NMF

NMF stands for non-negative matrix factorization, a technique for obtaining low rank representation of matrices with non-negative or positive elements

NMF decomposes multivariate data by creating a user-defined number of features. Each feature is a linear combination of the original attribute set; the coefficients of these linear combinations are non-negative.

NMF decomposes a data matrix V into the product of two lower rank matrices W and H so that V is approximately equal to W times H. NMF uses an iterative procedure to modify the initial values of W and H so that the product approaches V. The procedure terminates when the approximation error converges or the specified number of iterations is reached.

During model apply, an NMF model maps the original data into the new set of attributes (features) discovered by the model.

In [ ]:
from surprise import NMF
#fitting the model using the NMF algorithm with the prepared trainset data from above chuck of code.
nmf = NMF()
nmf.fit(trainset)

### SVD++

The SVD++ algorithm, an extension of SVD taking into account implicit ratings.


- __ Predicted Rating : __
    - 
    - $ \large \hat{r}_{ui} = \mu + b_u + b_i + q_i^T\left(p_u +
    |I_u|^{-\frac{1}{2}} \sum_{j \in I_u}y_j\right) $ 

- $ \pmb{I_u}$ --- the set of all items rated by user u

- $\pmb{y_j}$ --- Our new set of item factors that capture implicit ratings.

- __Optimization problem with user item interactions and regularization (to avoid overfitting)__
    - 
    - $ \large \sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
\lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2 + ||y_j||^2\right) $ 

In [ ]:
#fitting the model using the SVD++ algorithm with the prepared trainset data from above chuck of code.
svdpp = SVDpp()
svdpp.fit(trainset)

- cross validation

In [ ]:
cross_validate(svd, data,measures=['RMSE'], cv = 5, verbose = True, n_jobs=-1)

In [ ]:
cross_validate(nmf, data,measures=['RMSE'], cv = 5, verbose = True, n_jobs=-1) 

In [ ]:
cross_validate(svdpp, data,measures=['RMSE'], cv = 5, verbose = True,n_jobs=-1) 

- Prediction

In [ ]:
"""
   This function takes an model, dataframe(df) which is the test, user id(uid) from the dataframe and item id(iid)
   which in our case is movie id. It takes the test data and fit the model to make prediction of the ratings.
   tqm is to see the prediction time or how the function has progressed in the data.
"""

def predicions(model, df, uid = 'userId', iid = 'movieId'):
    from tqdm import tqdm
    
    return[
        model.predict(uid=data[uid],iid =data[iid])[3]
        for _, data in tqdm(test[[uid,iid]].iterrows(),total=len(test))
    ]

In [ ]:
#using above function to make predictions with the svd model
pred= predicions(svd, test, uid = 'userId', iid = 'movieId')

In [ ]:
#using above function to make predictions with the nmf model
pred0 = predicions(nmf, test, uid = 'userId', iid = 'movieId')

In [ ]:
#using above function to make predictions with the svdpp model
pred1 = predicions(svdpp, test, uid = 'userId', iid = 'movieId')

- Submission

In [ ]:
"""This code creates a new column on the test dataframe with the userid and movieid merged together """

test['Id'] = test['userId'].map(str)+ "_" +test['movieId'].map(str)
test.head()

In [ ]:
#Sellecting the index of the test dataframe
final_test= test["Id"]

In [ ]:
#Creating the submission Dataframe
Final_Table = {'Id': final_test, 'rating':np.round(pred, 1)}
submission = pd.DataFrame(data=Final_Table)
submission = submission[['Id', 'rating']]

In [ ]:
#Creating the submission Dataframe
Final_Table0 = {'Id': final_test, 'rating':np.round(pred0, 1)}
submission0 = pd.DataFrame(data=Final_Table)
submission0 = submission0[['Id', 'rating']]

In [ ]:
#Creating the submission Dataframe
Final_Table1 = {'Id': final_test, 'rating':np.round(pred1, 1)}
submission1 = pd.DataFrame(data=Final_Table)
submission1 = submission1[['Id', 'rating']]

- saving submission to csv files

In [ ]:
submission.to_csv("SvdSubmission.csv",index  = False) #writing csv file

In [ ]:
submission0.to_csv("nmfSubmission.csv",index  = False) #writing csv file

In [ ]:
submission1.to_csv("SvdppSubmission.csv",index  = False) #wrting csv file

In [ ]:
# This is for Comet purposes
#pre_comet = []
#pre_comet_nmf = []
pre_comet_svd = []
for _, row in test_data.iterrows():
    x_unseen = svd.predict(uid=row['userId'],iid=row['movieId'])
    pred = x_unseen[3]
    pre_comet_svd.append(pred)

In [ ]:
# Log metrics

#experiment = Experiment(api_key="PXaqoLTlHzWF85kimEv0zHJPz")
with experiment.context_manager("validate"):
    #svd.fit(trainset)
    #rmse_svdpp = mean_squared_error(test_data['rating'],pre_comet, squared=False)
    #rmse_nmf = mean_squared_error(test_data['rating'],pre_comet_nmf, squared=False)
    rmse_svd = mean_squared_error(test_data['rating'],pre_comet_nmf, squared=False)
  # returns the validation accuracy
    experiment.log_metric("rmse", rmse_svd)

In [ ]:
# End the experiment

experiment.end()

In [ ]:
# Display comet experiment

experiment.display()

# CONCLUSION

The best score we got when we used 50% of the data was 0.815. This is the score we submitted on the leaderboard. The best score came from fitting the model SVD Ffrom the surprise library. 